## Example of python program converting a raw CSV file to a DataFrame and then exporting to a PostgresSQL table

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sqlalchemy import create_engine
import psycopg2 
import psycopg2.extras
import io
import getpass

In [3]:
# Data for the project comes from https://github.com/JeffSackmann/tennis_wta
path = 'https://raw.githubusercontent.com/JeffSackmann/tennis_wta/master'

In [4]:
# A list of the data for the WTA matches from 2010 to 2021 is created and then Concatenated into a DataFrame
df_list = [pd.read_csv(path + '/wta_matches_' + str(year) + '.csv') for year in range(2010,2021)]
df = pd.concat(df_list)
df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2010-D001,Fed Cup WG R1: UKR vs ITA,Hard,4,D,20100206,1,201416,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,26.0,2020.0,18.0,2720.0
1,2010-D001,Fed Cup WG R1: UKR vs ITA,Hard,4,D,20100206,2,201355,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,3160.0,33.0,1680.0
2,2010-D001,Fed Cup WG R1: UKR vs ITA,Hard,4,D,20100206,3,201355,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,3160.0,26.0,2020.0
3,2010-D001,Fed Cup WG R1: UKR vs ITA,Hard,4,D,20100206,4,201212,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,18.0,2720.0,33.0,1680.0
4,2010-D002,Fed Cup WG R1: CZE vs GER,Hard,4,D,20100206,1,201394,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,66.0,950.0,46.0,1325.0


In [5]:
df.shape

(29252, 49)

In [6]:
df_wta_matches = df[['tourney_date', 'tourney_name', 'tourney_level', 'surface', 'round', 'best_of', 'winner_id', 'winner_name', 'winner_rank', 'loser_id', 'loser_name', 'loser_rank']]
df_wta_matches.insert(loc=0, column='index', value=np.arange(len(df_wta_matches)))
df_wta_matches.head()

,index,tourney_date,tourney_name,tourney_level,surface,round,best_of,winner_id,winner_name,winner_rank,loser_id,loser_name,loser_rank
0,0,20100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201416,Alona Bondarenko,26.0,201212,Francesca Schiavone,18.0
1,1,20100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201355,Flavia Pennetta,12.0,201435,Kateryna Bondarenko,33.0
2,2,20100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201355,Flavia Pennetta,12.0,201416,Alona Bondarenko,26.0
3,3,20100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201212,Francesca Schiavone,18.0,201435,Kateryna Bondarenko,33.0
4,4,20100206,Fed Cup WG R1: CZE vs GER,D,Hard,RR,3,201394,Anna Lena Groenefeld,66.0,201425,Lucie Safarova,46.0


In [7]:
# All WTA players Column header added.
df_players = pd.read_csv(path + '/wta_players.csv', names=['player_id', 'first_name', 'last_name', 'hand', 'birth_date', 'country'])
df_players.head()

,player_id,first_name,last_name,hand,birth_date,country
0,113190,Bobby,Riggs,U,NaN,USA
1,200000,X,X,U,19000000.0,UNK
2,200001,Martina,Hingis,R,19800930.0,SUI
3,200002,Mirjana,Lucic,R,19820309.0,CRO
4,200003,Justine,Henin,R,19820601.0,BEL


In [8]:
df_players.shape

(41743, 6)

In [9]:
# Current WTA rankings.  Column header added.
df_rankings = pd.read_csv(path + '/wta_rankings_current.csv', names=['date', 'ranking', 'id', 'points', 'unk'])
df_rankings.insert(loc=0, column='index', value=np.arange(len(df_rankings)))
df_rankings.head()

,index,date,ranking,id,points,unk
0,0,20210104,1,202458,8717,17
1,1,20210104,2,201594,7255,17
2,2,20210104,3,211768,5780,16
3,3,20210104,4,213550,5760,25
4,4,20210104,5,202494,5260,26


In [10]:
df_rankings_wta = df_rankings[['index', 'date', 'ranking', 'id']]
df_rankings_wta.head()

,index,date,ranking,id
0,0,20210104,1,202458
1,1,20210104,2,201594
2,2,20210104,3,211768
3,3,20210104,4,213550
4,4,20210104,5,202494


In [11]:
df_rankings.shape

(44197, 6)

In [12]:
try:
    p = getpass.getpass()
except Exception as error:
    print('ERROR', error)

········


In [13]:
engine = create_engine('postgresql://postgres:' + p + '@localhost:5432/WTA')

#Add matches, players and rankings DataFrames to postgres Database WTA.  Replace table if table already exists
df_wta_matches.to_sql('matches', con=engine, if_exists='replace',index=False)
df_players.to_sql('players', con=engine, if_exists='replace',index=False)
df_rankings_wta.to_sql('rankings', con=engine, if_exists='replace',index=False)



# Example python program to read data from a PostgreSQL table and load into a pandas DataFrame

In [14]:
# Connect to PostgreSQL server
dbConnection = engine.connect()

In [15]:
# Read data from PostgreSQL database table and load into a DataFrame instance
dataFrame = pd.read_sql("select * from \"matches\"", dbConnection)

In [16]:
# Print the DataFrame
dataFrame.head()

,index,tourney_date,tourney_name,tourney_level,surface,round,best_of,winner_id,winner_name,winner_rank,loser_id,loser_name,loser_rank
0,0,20100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201416,Alona Bondarenko,26.0,201212,Francesca Schiavone,18.0
1,1,20100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201355,Flavia Pennetta,12.0,201435,Kateryna Bondarenko,33.0
2,2,20100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201355,Flavia Pennetta,12.0,201416,Alona Bondarenko,26.0
3,3,20100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201212,Francesca Schiavone,18.0,201435,Kateryna Bondarenko,33.0
4,4,20100206,Fed Cup WG R1: CZE vs GER,D,Hard,RR,3,201394,Anna Lena Groenefeld,66.0,201425,Lucie Safarova,46.0


In [17]:
# Close the database connection
dbConnection.close();

## Example of pre-processing data before plugging it on a Machine Learning model

In [18]:
dataFrame.shape

(29252, 13)

In [22]:
dataFrame.dtypes

index                     int64
tourney_date     datetime64[ns]
tourney_name             object
tourney_level            object
surface                  object
round                    object
best_of                   int64
winner_id                 int64
winner_name              object
winner_rank             float64
loser_id                  int64
loser_name               object
loser_rank              float64
dtype: object

In [19]:
# Converting tourney_date to datetime format
dataFrame['tourney_date'] = pd.to_datetime(dataFrame['tourney_date']) 
# Restricing dates
# df_atp = df_atp.loc[(df_atp['Date'] > '2014-11-09') & (df_atp['Date'] <= '2016-11-09')]
# Keeping only completed matches
# df_atp = df_atp[df_atp['Comment'] == 'Completed'].drop("Comment",axis = 1)
# Rename Best of to Best_of
# df_atp.rename(columns = {'Best of':'Best_of'},inplace=True)
# Choosing features
# cols_to_keep = ['tourney_date','Series','surface', 'round','best_of', 'winner_rank','loser_rank']
# Dropping NaN
dataFrame = dataFrame.dropna()
# Dropping errors in the dataset and unimportant entries (e.g. there are very few entries for Masters Cup)
# dataFrame = dataFrame[(dataFrame['loser_rank'] != 'NR') & (dataFrame['winner_rank'] != 'NR') & (dataFrame['Series'] != 'Masters Cup')]
dataFrame.head()

,index,tourney_date,tourney_name,tourney_level,surface,round,best_of,winner_id,winner_name,winner_rank,loser_id,loser_name,loser_rank
0,0,1970-01-01 00:00:00.020100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201416,Alona Bondarenko,26.0,201212,Francesca Schiavone,18.0
1,1,1970-01-01 00:00:00.020100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201355,Flavia Pennetta,12.0,201435,Kateryna Bondarenko,33.0
2,2,1970-01-01 00:00:00.020100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201355,Flavia Pennetta,12.0,201416,Alona Bondarenko,26.0
3,3,1970-01-01 00:00:00.020100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201212,Francesca Schiavone,18.0,201435,Kateryna Bondarenko,33.0
4,4,1970-01-01 00:00:00.020100206,Fed Cup WG R1: CZE vs GER,D,Hard,RR,3,201394,Anna Lena Groenefeld,66.0,201425,Lucie Safarova,46.0


In [20]:
dataFrame.shape

(28034, 13)

In [23]:
dataFrame.dtypes

index                     int64
tourney_date     datetime64[ns]
tourney_name             object
tourney_level            object
surface                  object
round                    object
best_of                   int64
winner_id                 int64
winner_name              object
winner_rank             float64
loser_id                  int64
loser_name               object
loser_rank              float64
dtype: object

In [24]:
# Transform strings into numerical values
dataFrame[['best_of','winner_rank','loser_rank']] = dataFrame[['best_of','winner_rank','loser_rank']].astype(int)
dataFrame.head()

,index,tourney_date,tourney_name,tourney_level,surface,round,best_of,winner_id,winner_name,winner_rank,loser_id,loser_name,loser_rank
0,0,1970-01-01 00:00:00.020100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201416,Alona Bondarenko,26,201212,Francesca Schiavone,18
1,1,1970-01-01 00:00:00.020100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201355,Flavia Pennetta,12,201435,Kateryna Bondarenko,33
2,2,1970-01-01 00:00:00.020100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201355,Flavia Pennetta,12,201416,Alona Bondarenko,26
3,3,1970-01-01 00:00:00.020100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201212,Francesca Schiavone,18,201435,Kateryna Bondarenko,33
4,4,1970-01-01 00:00:00.020100206,Fed Cup WG R1: CZE vs GER,D,Hard,RR,3,201394,Anna Lena Groenefeld,66,201425,Lucie Safarova,46


In [25]:
dataFrame.dtypes

index                     int64
tourney_date     datetime64[ns]
tourney_name             object
tourney_level            object
surface                  object
round                    object
best_of                   int32
winner_id                 int64
winner_name              object
winner_rank               int32
loser_id                  int64
loser_name               object
loser_rank                int32
dtype: object

In [26]:
# Create an extra columns for the variable win described above using an auxiliary function win(x)
def win(x):
    if x > 0:
        return 0
    elif x <= 0:
        return 1  
    
dataFrame['win'] = (dataFrame['winner_rank'] - dataFrame['loser_rank']).apply(win)
dataFrame.head()

,index,tourney_date,tourney_name,tourney_level,surface,round,best_of,winner_id,winner_name,winner_rank,loser_id,loser_name,loser_rank,win
0,0,1970-01-01 00:00:00.020100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201416,Alona Bondarenko,26,201212,Francesca Schiavone,18,0
1,1,1970-01-01 00:00:00.020100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201355,Flavia Pennetta,12,201435,Kateryna Bondarenko,33,1
2,2,1970-01-01 00:00:00.020100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201355,Flavia Pennetta,12,201416,Alona Bondarenko,26,1
3,3,1970-01-01 00:00:00.020100206,Fed Cup WG R1: UKR vs ITA,D,Hard,RR,3,201212,Francesca Schiavone,18,201435,Kateryna Bondarenko,33,1
4,4,1970-01-01 00:00:00.020100206,Fed Cup WG R1: CZE vs GER,D,Hard,RR,3,201394,Anna Lena Groenefeld,66,201425,Lucie Safarova,46,0


### Exploratory Analysis for surfaces

In [28]:
# Look at percentage of wins for each surface. 
win_by_Surface = pd.crosstab(dataFrame.win, dataFrame.surface).apply( lambda x: x/x.sum(), axis = 0 )
win_by_Surface

surface,Carpet,Clay,Grass,Hard
win,,,,
0,0.308725,0.359161,0.360981,0.345233
1,0.691275,0.640839,0.639019,0.654767
